In [1]:
import SimpleITK as sitk
import numpy as np
import trimesh
from tqdm import tqdm
from scipy.ndimage import gaussian_filter
from sklearn.manifold import Isomap
from pathlib import Path
from dataloaders import VIA11_Corrected_CS_Loader
import scipy.ndimage as ndimage
import open3d as o3d
import copy

import subprocess as sp

from shape_utils import interpolate_segmentation

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
data_path = Path('/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup')


In [3]:
cs_ds = VIA11_Corrected_CS_Loader(bv_good=True, corrected=True, all_bv=False, preload=True)
        # unravell all the sulci
sulci_list = cs_ds.get_subjects()

100%|██████████| 286/286 [00:11<00:00, 25.80it/s]


In [4]:
idx = 13
print(f'Average sulcus automatically selected is {sulci_list[idx][0]}_{sulci_list[idx][1]}.')

Average sulcus automatically selected is right_sub-via033.


In [5]:
average_sulcus = sulci_list[idx]
average_sulcus

['right',
 'sub-via033',
 'corrected',
 <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x7fabff17a490> >]

In [6]:
sub5_left = sulci_list[4]
sub5_left

['left',
 'sub-via005',
 'corrected',
 <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x7fabff17a8e0> >]

In [7]:
sulc2 = [x for x in sub5_left]

# need to flip if different hemispheres
if average_sulcus[0] != sulc2[0]:
    sulc2[3] = sitk.Flip(sulc2[3], [False, False, True])
    print('Flipped')

# register the sulci with ICP
s1_points = np.stack(np.where(sitk.GetArrayFromImage(average_sulcus[3]))).T
s2_points = np.stack(np.where(sitk.GetArrayFromImage(sulc2[3]))).T

# finds the transformation matrix sending a to b
trnsh_mat, transformed, __ = trimesh.registration.icp(a=s2_points,
                                                      b=s1_points,
                                                      max_iterations=1000,
                                                      scale=False,)

transformed_img = np.zeros_like(sitk.GetArrayFromImage(sulc2[3]))
transformed_img[tuple(np.round(transformed).astype(int).T)] = 1
transformed_img = sitk.GetImageFromArray(transformed_img.astype(np.float32))

res_sulci = [x for x in sulc2]
res_sulci[3] = transformed_img

Flipped


In [8]:
A_registered_to_B_interp = interpolate_segmentation(imageA=sitk.GetArrayFromImage(sulc2[3]),
                                                    imageB_shape=sitk.GetArrayFromImage(average_sulcus[3]).shape,
                                                    pointsAregistered2B=transformed,
                                                    trnsh_mat=trnsh_mat,
                                                    spline_order=1)

In [9]:
sub5REGTOavg = res_sulci[3]

In [10]:
i = sitk.ReadImage('/mnt/projects/VIA_Vlad/nobackup/BrainVisa/CS_edited/CFIN/LSulci_sub-via010_default_session_best_edit_NHT.nii.gz')
sub5REGTOavg.CopyInformation(i)
sub5REGTOavg = sitk.Cast(sub5REGTOavg, sitk.sitkInt16)

In [11]:
interp_transf = sitk.GetImageFromArray(A_registered_to_B_interp.astype(np.int16))
interp_transf.CopyInformation(i)

In [12]:
sitk.WriteImage(sub5REGTOavg, '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5REGTOavg.nii.gz')
sitk.WriteImage(average_sulcus[3], '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub17_right_orig.nii.gz')
sitk.WriteImage(sulc2[3], '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5_left_flipped.nii.gz')
sitk.WriteImage(sub5_left[3], '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5_left_orig.nii.gz')
sitk.WriteImage(interp_transf, '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5_left_interp.nii.gz')

In [13]:
import subprocess as sp

In [14]:
cmd = ['/mrhome/vladyslavz/portable_apps/brainvisa4_5/bin/AimsMeshBrain', '-i', '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5_left_interp.nii.gz', '-o', '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5_left_interp.ply']
sp.run(cmd)

reading image      : done
resolutions        : (0.899994 mm,0.902778 mm,0.902771 mm)
reading slice      :                      1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 91010101010101010101011111111111111111111121212121212121212121313131313131313131314141414141414141414151515151515151515151616161616161616161617171717171717171717181818181818181818181919191919191919191920202020202020202020212121212121212121212222222222222222222223232323232323232323242424242424242424242525252525252525252526262626262626262626272727272727272727272828282828282828288
getting interface  :         done 
processing mesh    : reduced neighborhoodextended neighborhoovertices            smoothing  iteration                                            1   1   1   1   1   1   1   1   1   1   2   2   2   2   2   2   2   2   2   2   3                       pass 1/3 : decimating      

CompletedProcess(args=['/mrhome/vladyslavz/portable_apps/brainvisa4_5/bin/AimsMeshBrain', '-i', '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5_left_interp.nii.gz', '-o', '/mrhome/vladyslavz/git/central-sulcus-analysis/shape_features/data/nobackup/sub5_left_interp.ply'], returncode=0)